# PYNTCLOUD POINT CLOUD VIEWING UTILITY

This utility enables you to plot generate point clouds from a '.raw' file using both ndarrays and dataframes. The standard backend is either three.js or pythree.js,the latter allowing you to view all point-clouds within your jupyter notebook. You can save your point cloud as a '.ply'.

In [1]:
import numpy as np
import pandas as pd
import os
from pyntcloud import PyntCloud
import matplotlib
import matplotlib.pyplot as plt
import math
import subprocess
import glob
import time

### map_to_plane
Takes in the x and y indeces of the ndarray, 
as well as the depth values and determines the corrected 
x position, y position and depth based on the focal length 
of the camera.

In [2]:
def map_to_plane(x_index, y_index, tof_depth, width, height):
    # Compute d
    focal_length = 451.19988773439741
    x_center = (width - 1) / 2
    y_center = (height - 1) / 2
    delta_x = (x_index - x_center)
    delta_y = (y_index - y_center)
    d = math.sqrt(delta_x**2 + delta_y**2)
    
    # Compute lens angle and plane angle
    theta = math.atan(d/focal_length)
    phi = math.atan(delta_x/delta_y)
    
    # Compute planar coords
    planar_z = math.cos(theta) * tof_depth
    scaling_factor = (-1) * planar_z / focal_length
    planar_x = scaling_factor * delta_x
    planar_y = scaling_factor * delta_y
    
    return planar_x, planar_y, planar_z

### raw_to_ndarray
Takes a filename and width, reads in data 
from a '.raw' file into a 1D ndarray, reshapes the array 
based on width,uses MAP_TO_PLANE to compute the x, y and z 
position of each point and stores them in a 3 column array.

In [3]:
def raw_to_ndarray(depth_file, width):
    # Read file into ndarray
    depth_list = np.fromfile(depth_file, dtype='float32')
    # Reshape ndarray
    depth_map = np.reshape(depth_list, (-1, width))
    height = depth_map.size / width
    # Allocate points ndarray
    np_points = np.zeros(shape=(depth_list.size, 3))
    gen_pos = 0
    # Fill points ndarray
    for index, depth in np.ndenumerate(depth_map):
        x_pos, y_pos, z_pos = map_to_plane(index[1],
                                           index[0],
                                           depth,
                                           width,
                                           height)
        np_points[gen_pos] = [x_pos, z_pos, y_pos]
        gen_pos += 1
    
    np_points, discarded_points = np.vsplit(np_points, np.array([480 * width]))
    return np_points

### get_point_cloud
Takes in a 3-column ndarray and casts it into a pandas dataframe with column labels ['x', 'y', 'z'],then uses the Pyntcloud library to instantiate a pyntcloud object from the dataframe.

In [4]:
def get_point_cloud(np_points):
    # Convert to pandas data frame
    pandas_points = pd.DataFrame(np_points, columns=['x', 'y', 'z'])
    # Construct cloud from data frame
    cloud = PyntCloud(pandas_points)
    return cloud

### filter_cloud
Saves pre-filtered state as a scene, then 
applies the specified filter with the variables defined 
by val_1 and val_2.

In [5]:
def filter_cloud(cloud, filt_type, val_1, val_2):
    # Save state before filter
    scene = cloud.plot(backend="pythreejs", initial_point_size=10, 
                   use_as_color="z", cmap="Wistia_r", return_scene=True)
    # Apply KDTree filter
    kdtree_id = cloud.add_structure("kdtree")
    if filt_type == 'SOR':
        cloud.get_filter("SOR", and_apply=True, kdtree_id=kdtree_id, k=val_1, z_max=val_2)
    elif filt_type == 'ROR':
        cloud.get_filter("ROR", and_apply=True, kdtree_id=kdtree_id, k=val_1, r=val_2)
    else:
        print('Invalid filter type.\nValid options are:\nSOR\nROR')
    
    # Return prior state
    return scene

### x_plot
Cross-plots two point clouds in the same window using both a cloud and a saved scene. Also accepts an offset axis that allows point clouds to be positioned next to each other rather than overlaid.

In [6]:
def x_plot(scene, cloud_2, offset = None):
    
    if offset is 'x':
        cloud.xyz[:,0] += 2 * np.amax(cloud.xyz[:,0]) + 1000
    elif offset is 'y':
        cloud.xyz[:,1] += 2 * np.amax(cloud.xyz[:,1]) + 1000
    elif offset is 'z':
        cloud.xyz[:,2] += np.amax(cloud.xyz[:,2])
    else:
        pass
    
    cloud_2.plot(backend="pythreejs", initial_point_size=10, 
             use_as_color="z", cmap="cool_r", scene=scene)


### get_io
Determines input and output files using a string 
of 4 ints 'xxxx' and calls RAW_TO_NDARRAY. Returns both the
3-column ndarray and the output file path.

In [7]:
def get_io(file_path, width):
    base_name = os.path.basename(file_path)
    file_name, ext = os.path.splitext(base_name)
    dest_path = os.path.join('ply_files', file_name + '.ply')
    if not os.path.isdir('ply_files'):
        subprocess.check_call(['mkdir', 'ply_files'])
    np_points = raw_to_ndarray(file_path, width)
    return np_points, dest_path

### get_lines
Plots a red, a green and a blue line corresponding to the x, y and z axes respectively centered around the origin

In [8]:
def get_lines():
    scaling = 2**14
    horiz_fov = 74
    vert_fov = 49
    x = scaling * math.sin(horiz_fov*math.pi/360)
    z = scaling * math.tan(vert_fov*math.pi/360)
    y = scaling * math.cos(horiz_fov*math.pi/360)
    
    corner_1 = [x,y,z]
    corner_2 = [-x,y,z]
    corner_3 = [-x,y,-z]
    corner_4 = [x,y,-z]
    
    lines = [
        # X,Y,Z Axes lines
        {
            "color": "red",
            "vertices": [[0,0,0], [1000,0,0]]
        },
        {
            "color": "blue",
            "vertices": [[0,0,0], [0,1000,0]]
        },
        {
            "color": "green",
            "vertices": [[0,0,0], [0,0,1000]]
        },
        # FOV Lines
        {
            "color": "white",
            "vertices": [[0,0,0], [x, y, z]]
        },
        {
            "color": "white",
            "vertices": [[0,0,0], [-x, y, z]]
        },
        {
            "color": "white",
            "vertices": [[0,0,0], [x, y, -z]]
        },
        {
            "color": "white",
            "vertices": [[0,0,0], [-x, y, -z]]
        },
        # Frustums
        {
            "color": "white",
            "vertices": [corner_1, corner_2, corner_3, corner_4, corner_1]
        },
        {
            "color": "gray",
            "vertices": [[i * 0.25 for i in corner_1],
                         [i * 0.25 for i in corner_2], 
                         [i * 0.25 for i in corner_3],
                         [i * 0.25 for i in corner_4],
                         [i * 0.25 for i in corner_1]]
        },
        {
            "color": "gray",
            "vertices": [[i * 0.50 for i in corner_1],
                         [i * 0.50 for i in corner_2], 
                         [i * 0.50 for i in corner_3],
                         [i * 0.50 for i in corner_4],
                         [i * 0.50 for i in corner_1]]
        },
        {
            "color": "gray",
            "vertices": [[i * 0.75 for i in corner_1],
                         [i * 0.75 for i in corner_2], 
                         [i * 0.75 for i in corner_3],
                         [i * 0.75 for i in corner_4],
                         [i * 0.75 for i in corner_1]]
        }
    ]
    
    
    
    return lines

### Controls for Point Cloud Viewer:
#### Zoom: `Scroll` | Orbit: `Left Click and Drag` | Shift: `Right Click and Drag`

### Get point cloud from `.raw`

In [9]:
#raw_path = '/Users/crog/Documents/tof_proc_lib_ap/test_data/ref_image_files/Man_Scene_Data'
raw_path = '/Users/crog/Documents/pointcloud/depth_output_files'
raw_file = os.path.join(raw_path, 'depth_output_0009.raw')
np_points, dest_path = get_io(raw_file, 668)
cloud = get_point_cloud(np_points)
lines = get_lines()

### Plot unfiltered point cloud with pythreejs backend

In [10]:
cloud.plot(backend="pythreejs", initial_point_size=10, use_as_color="y", cmap="viridis_r", polylines=lines)

/anaconda3/envs/pyntcloud/lib/python3.6/site-packages/pythreejs/traits.py:177: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, far=16777216.0, fov=90.0, position=(0.0, -10.0, 0.0), quaternion…

### Plot unfiltered point cloud with threejs backend

In [29]:
cloud.plot(backend="threejs", initial_point_size=10, use_as_color="y", cmap="viridis_r")
# Move files into folder: threejs_plot
if os.path.isdir('threejs_tmp'):
    subprocess.check_call(['rm', '-rf', 'threejs_tmp'])
subprocess.check_call(['mkdir', 'threejs_tmp'])
files = glob.glob('pyntcloud_plot*')
for file in files:
    subprocess.check_call(['mv', file, 'threejs_tmp/.'])

# Open html file:
subprocess.check_call(['open', 'threejs_tmp/pyntcloud_plot.html'])

# Clean up
time.sleep(5)
subprocess.check_call(['rm', '-rf', 'threejs_tmp'])

0

### Cross plot filtered and unfiltered point-cloud with pythreejs backend

In [ ]:
scene = filter_cloud(cloud, 'SOR', 1.5, 0.5)
x_plot(scene, cloud, offset='x')

### Save point-cloud to `.ply`

In [ ]:
cloud.to_file(dest_path)